In [1]:
from pynq import Overlay
overlay = Overlay('overlay/bram_native_bd.bit')

In [2]:
from pynq import MMIO

In [3]:
BRAM_BASE_ADDRESS=0xC0020000
BRAM_ADDRESS_RANGE=128*1024

In [4]:
def list_comp(l1,l2,length):
    for i in range(length):
        if l1[i] != l2[i]:
            return False
    return True

In [5]:
from enum import IntEnum
import time
from pynq import allocate
import numpy as np

In [6]:
buffer_len = 16*1024
input_buffer = allocate(shape=(buffer_len,), dtype=np.uint64)

for i in range(buffer_len):
   input_buffer[i] = i+0x0000_0020_0000_0000
#    input_buffer[i] = i

In [7]:
print("Input:")

for i in range(10):
    print(str(input_buffer[i]) + ", ", end="")
print()

Input:
137438953472, 137438953473, 137438953474, 137438953475, 137438953476, 137438953477, 137438953478, 137438953479, 137438953480, 137438953481, 


In [8]:
def getbit(value, order):
    orderVal = 2**order
    tmpbin = value & orderVal
    if (tmpbin):
        return 1
    else:
        return 0

def changebit(value, order, bit):
    if (bit == 1):
        return value | (1 << order)
    else:
        return value & ~(1 << order)

class Cdma:
    CDMACR = 0x0
    CDMASR = 0x4
    SA = 0x18
    DA = 0x20
    BTT = 0x28
    
    def __init__(self, overlay):
        self.cdma = overlay.axi_cdma_0
    
    def read(self, address):
        return self.cdma.read(address)
        
    def write(self, address, value):
        self.cdma.write(address, value)

    def transfer(self, src, dst, size):
        # Step 1
        cdmasr = self.read(self.CDMASR)
        cdmasrIdle = getbit(cdmasr, 1)
        if (cdmasrIdle != 1):
            print("CDMA is busy..")
            return

        # Step 2
        cdmacr = self.read(self.CDMACR)
        cdmacr = changebit(cdmacr, 12, 1) # set IOC_IrqEn
        cdmacr = changebit(cdmacr, 14, 1) # set ERR_IrqEn
        self.write(self.CDMACR, cdmacr)

        # Step 3
        self.write(self.SA, src)

        # Step 4
        self.write(self.DA, dst)

        # Step 5
        self.write(self.BTT, size)

        # Step 6
        self.read(self.CDMASR)
        cdmasrIdle = getbit(cdmasr, 1)
        while (cdmasrIdle != 1):
            print(".", end="")
            self.read(self.CDMASR)
            cdmasrIdle = getbit(cdmasr, 1)

        # Step 7-8
        cdmasr = self.read(self.CDMASR)
        cdmasr = changebit(cdmasr, 12, 1) # clear IOC_Irq
        self.write(self.CDMASR, cdmasr)

        print("Transfered " + str(size) + " bytes from " + "0x{:08x}".format(src) + " to " + "0x{:08x}".format(dst))

    def reset(self):
        cdmacr = self.read(self.CDMACR)
        cdmacr = changebit(cdmacr, 2, 1)
        self.write(self.CDMACR, cdmacr)
        print("Reset CDMA")

In [9]:
cdma_obj = Cdma(overlay)

In [10]:
output_buffer = allocate(shape=(buffer_len,), dtype=np.uint64)

In [14]:
print("==== READ CDMA ====")

# start_time = time.time()
cdma_obj.transfer(BRAM_BASE_ADDRESS, output_buffer.physical_address, buffer_len*8)
# end_time = time.time()

==== READ CDMA ====
Transfered 131072 bytes from 0xc0020000 to 0x168e0000


In [15]:
for i in range(buffer_len):
   print(i,output_buffer[i])

0 137438953472
1 137438953473
2 137438953474
3 137438953475
4 137438953476
5 137438953477
6 137438953478
7 137438953479
8 137438953480
9 137438953481
10 137438953482
11 137438953483
12 137438953484
13 137438953485
14 137438953486
15 137438953487
16 137438953488
17 137438953489
18 137438953490
19 137438953491
20 137438953492
21 137438953493
22 137438953494
23 137438953495
24 137438953496
25 137438953497
26 137438953498
27 137438953499
28 137438953500
29 137438953501
30 137438953502
31 137438953503
32 137438953504
33 137438953505
34 137438953506
35 137438953507
36 137438953508
37 137438953509
38 137438953510
39 137438953511
40 137438953512
41 137438953513
42 137438953514
43 137438953515
44 137438953516
45 137438953517
46 137438953518
47 137438953519
48 137438953520
49 137438953521
50 137438953522
51 137438953523
52 137438953524
53 137438953525
54 137438953526
55 137438953527
56 137438953528
57 137438953529
58 137438953530
59 137438953531
60 137438953532
61 137438953533
62 137438953534
63

541 137438954013
542 137438954014
543 137438954015
544 137438954016
545 137438954017
546 137438954018
547 137438954019
548 137438954020
549 137438954021
550 137438954022
551 137438954023
552 137438954024
553 137438954025
554 137438954026
555 137438954027
556 137438954028
557 137438954029
558 137438954030
559 137438954031
560 137438954032
561 137438954033
562 137438954034
563 137438954035
564 137438954036
565 137438954037
566 137438954038
567 137438954039
568 137438954040
569 137438954041
570 137438954042
571 137438954043
572 137438954044
573 137438954045
574 137438954046
575 137438954047
576 137438954048
577 137438954049
578 137438954050
579 137438954051
580 137438954052
581 137438954053
582 137438954054
583 137438954055
584 137438954056
585 137438954057
586 137438954058
587 137438954059
588 137438954060
589 137438954061
590 137438954062
591 137438954063
592 137438954064
593 137438954065
594 137438954066
595 137438954067
596 137438954068
597 137438954069
598 137438954070
599 1374389540

1040 137438954512
1041 137438954513
1042 137438954514
1043 137438954515
1044 137438954516
1045 137438954517
1046 137438954518
1047 137438954519
1048 137438954520
1049 137438954521
1050 137438954522
1051 137438954523
1052 137438954524
1053 137438954525
1054 137438954526
1055 137438954527
1056 137438954528
1057 137438954529
1058 137438954530
1059 137438954531
1060 137438954532
1061 137438954533
1062 137438954534
1063 137438954535
1064 137438954536
1065 137438954537
1066 137438954538
1067 137438954539
1068 137438954540
1069 137438954541
1070 137438954542
1071 137438954543
1072 137438954544
1073 137438954545
1074 137438954546
1075 137438954547
1076 137438954548
1077 137438954549
1078 137438954550
1079 137438954551
1080 137438954552
1081 137438954553
1082 137438954554
1083 137438954555
1084 137438954556
1085 137438954557
1086 137438954558
1087 137438954559
1088 137438954560
1089 137438954561
1090 137438954562
1091 137438954563
1092 137438954564
1093 137438954565
1094 137438954566
1095 13743

1540 137438955012
1541 137438955013
1542 137438955014
1543 137438955015
1544 137438955016
1545 137438955017
1546 137438955018
1547 137438955019
1548 137438955020
1549 137438955021
1550 137438955022
1551 137438955023
1552 137438955024
1553 137438955025
1554 137438955026
1555 137438955027
1556 137438955028
1557 137438955029
1558 137438955030
1559 137438955031
1560 137438955032
1561 137438955033
1562 137438955034
1563 137438955035
1564 137438955036
1565 137438955037
1566 137438955038
1567 137438955039
1568 137438955040
1569 137438955041
1570 137438955042
1571 137438955043
1572 137438955044
1573 137438955045
1574 137438955046
1575 137438955047
1576 137438955048
1577 137438955049
1578 137438955050
1579 137438955051
1580 137438955052
1581 137438955053
1582 137438955054
1583 137438955055
1584 137438955056
1585 137438955057
1586 137438955058
1587 137438955059
1588 137438955060
1589 137438955061
1590 137438955062
1591 137438955063
1592 137438955064
1593 137438955065
1594 137438955066
1595 13743

2039 137438955511
2040 137438955512
2041 137438955513
2042 137438955514
2043 137438955515
2044 137438955516
2045 137438955517
2046 137438955518
2047 137438955519
2048 137438955520
2049 137438955521
2050 137438955522
2051 137438955523
2052 137438955524
2053 137438955525
2054 137438955526
2055 137438955527
2056 137438955528
2057 137438955529
2058 137438955530
2059 137438955531
2060 137438955532
2061 137438955533
2062 137438955534
2063 137438955535
2064 137438955536
2065 137438955537
2066 137438955538
2067 137438955539
2068 137438955540
2069 137438955541
2070 137438955542
2071 137438955543
2072 137438955544
2073 137438955545
2074 137438955546
2075 137438955547
2076 137438955548
2077 137438955549
2078 137438955550
2079 137438955551
2080 137438955552
2081 137438955553
2082 137438955554
2083 137438955555
2084 137438955556
2085 137438955557
2086 137438955558
2087 137438955559
2088 137438955560
2089 137438955561
2090 137438955562
2091 137438955563
2092 137438955564
2093 137438955565
2094 13743

...

14027 137438967499
14028 137438967500
14029 137438967501
14030 137438967502
14031 137438967503
14032 137438967504
14033 137438967505
14034 137438967506
14035 137438967507
14036 137438967508
14037 137438967509
14038 137438967510
14039 137438967511
14040 137438967512
14041 137438967513
14042 137438967514
14043 137438967515
14044 137438967516
14045 137438967517
14046 137438967518
14047 137438967519
14048 137438967520
14049 137438967521
14050 137438967522
14051 137438967523
14052 137438967524
14053 137438967525
14054 137438967526
14055 137438967527
14056 137438967528
14057 137438967529
14058 137438967530
14059 137438967531
14060 137438967532
14061 137438967533
14062 137438967534
14063 137438967535
14064 137438967536
14065 137438967537
14066 137438967538
14067 137438967539
14068 137438967540
14069 137438967541
14070 137438967542
14071 137438967543
14072 137438967544
14073 137438967545
14074 137438967546
14075 137438967547
14076 137438967548
14077 137438967549
14078 137438967550
14079 137438

14527 137438967999
14528 137438968000
14529 137438968001
14530 137438968002
14531 137438968003
14532 137438968004
14533 137438968005
14534 137438968006
14535 137438968007
14536 137438968008
14537 137438968009
14538 137438968010
14539 137438968011
14540 137438968012
14541 137438968013
14542 137438968014
14543 137438968015
14544 137438968016
14545 137438968017
14546 137438968018
14547 137438968019
14548 137438968020
14549 137438968021
14550 137438968022
14551 137438968023
14552 137438968024
14553 137438968025
14554 137438968026
14555 137438968027
14556 137438968028
14557 137438968029
14558 137438968030
14559 137438968031
14560 137438968032
14561 137438968033
14562 137438968034
14563 137438968035
14564 137438968036
14565 137438968037
14566 137438968038
14567 137438968039
14568 137438968040
14569 137438968041
14570 137438968042
14571 137438968043
14572 137438968044
14573 137438968045
14574 137438968046
14575 137438968047
14576 137438968048
14577 137438968049
14578 137438968050
14579 137438

15026 137438968498
15027 137438968499
15028 137438968500
15029 137438968501
15030 137438968502
15031 137438968503
15032 137438968504
15033 137438968505
15034 137438968506
15035 137438968507
15036 137438968508
15037 137438968509
15038 137438968510
15039 137438968511
15040 137438968512
15041 137438968513
15042 137438968514
15043 137438968515
15044 137438968516
15045 137438968517
15046 137438968518
15047 137438968519
15048 137438968520
15049 137438968521
15050 137438968522
15051 137438968523
15052 137438968524
15053 137438968525
15054 137438968526
15055 137438968527
15056 137438968528
15057 137438968529
15058 137438968530
15059 137438968531
15060 137438968532
15061 137438968533
15062 137438968534
15063 137438968535
15064 137438968536
15065 137438968537
15066 137438968538
15067 137438968539
15068 137438968540
15069 137438968541
15070 137438968542
15071 137438968543
15072 137438968544
15073 137438968545
15074 137438968546
15075 137438968547
15076 137438968548
15077 137438968549
15078 137438

15526 137438968998
15527 137438968999
15528 137438969000
15529 137438969001
15530 137438969002
15531 137438969003
15532 137438969004
15533 137438969005
15534 137438969006
15535 137438969007
15536 137438969008
15537 137438969009
15538 137438969010
15539 137438969011
15540 137438969012
15541 137438969013
15542 137438969014
15543 137438969015
15544 137438969016
15545 137438969017
15546 137438969018
15547 137438969019
15548 137438969020
15549 137438969021
15550 137438969022
15551 137438969023
15552 137438969024
15553 137438969025
15554 137438969026
15555 137438969027
15556 137438969028
15557 137438969029
15558 137438969030
15559 137438969031
15560 137438969032
15561 137438969033
15562 137438969034
15563 137438969035
15564 137438969036
15565 137438969037
15566 137438969038
15567 137438969039
15568 137438969040
15569 137438969041
15570 137438969042
15571 137438969043
15572 137438969044
15573 137438969045
15574 137438969046
15575 137438969047
15576 137438969048
15577 137438969049
15578 137438

16025 137438969497
16026 137438969498
16027 137438969499
16028 137438969500
16029 137438969501
16030 137438969502
16031 137438969503
16032 137438969504
16033 137438969505
16034 137438969506
16035 137438969507
16036 137438969508
16037 137438969509
16038 137438969510
16039 137438969511
16040 137438969512
16041 137438969513
16042 137438969514
16043 137438969515
16044 137438969516
16045 137438969517
16046 137438969518
16047 137438969519
16048 137438969520
16049 137438969521
16050 137438969522
16051 137438969523
16052 137438969524
16053 137438969525
16054 137438969526
16055 137438969527
16056 137438969528
16057 137438969529
16058 137438969530
16059 137438969531
16060 137438969532
16061 137438969533
16062 137438969534
16063 137438969535
16064 137438969536
16065 137438969537
16066 137438969538
16067 137438969539
16068 137438969540
16069 137438969541
16070 137438969542
16071 137438969543
16072 137438969544
16073 137438969545
16074 137438969546
16075 137438969547
16076 137438969548
16077 137438

In [45]:
print(list_comp(output_buffer, range(0x0000_0020_0000_0000, 0x0000_0020_0000_0000+buffer_len), buffer_len))

False


In [13]:
print("==== Test CDMA ====")

# start_time = time.time()
cdma_obj.transfer(input_buffer.physical_address, BRAM_BASE_ADDRESS, buffer_len*8)
# end_time = time.time()

# print("%s seconds" % (end_time - start_time))
# print()


==== Test CDMA ====
Transfered 131072 bytes from 0x16860000 to 0xc0020000
